# Feature Selection Methods project - Test Environment
<b>Author: Yuval Uner</b><br/>
<br/>
In this notebook, you can use and experiment with the feature selection tools developed by me for the project.<br/>
You can select a dataset, then try out and compare the different feature selection methods, and see how they affect the model's performance.<br/>

# Importing libraries

Import all of the libraries required for the project, both external libraries as well as the local libraries developed for the project.

## External libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error, max_error, log_loss, accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SequentialFeatureSelector, VarianceThreshold, SelectKBest, mutual_info_regression
import seaborn as sns

## Local libraries

In [ ]:
from data_pre_processing import PreprocessingPipeline
from filter_methods import WeightedCombination, VarianceInflationFactor
from deep_network_methods import LinearAgent, SequentialAgent
from shap_values_methods import BaseMethod, BranchingVariant, BacktrackingVariant

# Select the dataset, preprocess the data, and choose the model and metrics
Select one of the datasets below, load the data and and preprocess it, then select the relevant model and metrics to use for the task. <br/>
The datasets are divided into 2 categories: linear regression datasets and classification datasets. <br/>
For each category, there are 2 datasets to choose from. <br/>
Please choose only one dataset, and follow the model selection and metric selection steps of the category it belongs to.<br/>
<br/>
All datasets go through the same preprocessing pipeline, which performs the following (in order):
1. Convert categorical features to one-hot encoding
2. Impute missing values by replacing them with the mean of the column
3. Remove features with high collinearity between them. By default, this threshold is set to 0.9.
At the end of the pipeline, the preprocessed data is returned, along with a list of which of the remaining columns are categorical and which are numerical.<br/>

The data can then optionally be normalized using sklearn's StandardScaler.<br/>

## Linear Regression datasets
You may choose of either of the 2 datasets below:
1. Housing dataset. This is the dataset used in class. The same 10 features as used in the introduction lecture are used, as otherwise the dataset has too many features to handle in a reasonable amount of time by the more time consuming methods.
2. WHO life expectancy (https://www.kaggle.com/datasets/kumarajarshi/life-expectancy-who). A dataset containing 21 features (20 in this project, as 1 is always dropped) and 2938 rows. It features statistical information about countries and health factors in those countries over several years, with the target variable being the life expectancy in the country.

### Selecting the dataset - please choose one of the following datasets

#### Housing dataset

Load the dataset and select only the relevant columns

In [ ]:
data = pd.read_csv('data/data_houses.csv', index_col="Id")
cols = ["OverallQual","GrLivArea","GarageCars","GarageArea","TotalBsmtSF","FullBath","YearBuilt","YearRemodAdd",
        "LotFrontage","MSSubClass", "SalePrice"]
data = data[cols]
data

Set the target variable

In [ ]:
target = 'SalePrice'

For using the sequential agent - select the feature by which to sequence the data. This can be skipped if not using the sequential agent.<br/>
In this case, the feature chosen is YearBuilt, as it may make sense that houses built in different years may have different distributions of features and different sale price distributions.

In [ ]:
sequence_by = 'YearBuilt'

Next, preprocess the data using the preprocessing pipeline.

In [ ]:
preprocessor = PreprocessingPipeline(data, ['MSSubClass', 'FullBath'], 0.9, target_name='SalePrice')

In [ ]:
data, continuous_cols, cat_cols = preprocessor.preprocess()

#### WHO Life Expectancy

Load the dataset

In [ ]:
data = pd.read_csv('data/Life Expectancy Data.csv')
# Drop the country column, because it is categorical and with too many unique values.
data = data.drop(columns=['Country'])
data

Set the target variable

In [ ]:
target = 'Life expectancy '

For using the sequential agent - select the feature by which to sequence the data. This can be skipped if not using the sequential agent.<br/>
In this case, the feature chosen is Year, as it may make sense for distributions to change over the years, and for worldwide trends to be reflected in the progress of years.

In [ ]:
sequence_by = 'Year'

Next, preprocess the data using the preprocessing pipeline.

In [ ]:
preprocessor = PreprocessingPipeline(data,
                                     [
                                         'Status'
                                     ],
                                     0.9, target_name=target)

In [ ]:
data, continuous_cols, cat_cols = preprocessor.preprocess()

### Model and metric selection
For the linear regression datasets, we use a linear regression model, MSE as the loss function, as well as R2, MAE, Mean Average Percent Error and max error as the evaluation metrics.

In [ ]:
model = LinearRegression()
loss_func = mean_squared_error
metrics = [r2_score, mean_absolute_error, mean_absolute_percentage_error, max_error]

## Classification datasets (Logistic Regression)

You may choose of either of the 2 datasets below:
1. German credit dataset (https://www.kaggle.com/datasets/mpwolke/cusersmarildownloadsgermancsv). A dataset containing 20 features and 1000 rows. It contains information about credit applicants, with the target variable being whether the applicant is creditworthy or not.
2. Titanic dataset, imported via the seaborn library. A classic dataset, containing 14 features and 891 rows. It contains information about passengers on the Titanic, with the target variable being whether the passenger survived or not.

### Selecting the dataset - please choose one of the following datasets

#### Telco customer churn dataset

Load the dataset

In [ ]:
data = pd.read_csv('data/german.csv', sep=';')
data

Set the target variable

In [ ]:
target = 'Creditability'

For using the sequential agent - select the feature by which to sequence the data. This can be skipped if not using the sequential agent.<br/>
In this case, the feature chosen is tenure, Age_years, as it may make sense for the features and creditability to change over the years.

In [ ]:
sequence_by = 'Age_years'

Next, preprocess the data using the preprocessing pipeline.

In [ ]:
preprocessor = PreprocessingPipeline(data,
                                     cat_cols=[
                                         'Account_Balance', 'Payment_Status_of_Previous_Credit', 'Purpose', 'Value_Savings_Stocks',
                                         'Sex_Marital_Status', 'Most_valuable_available_asset',
                                         'Type_of_apartment', 'Occupation', 'Telephone', 'Foreign_Worker'
                                     ],
                                     threshold=0.9, target_name=target)
data, continuous_cols, cat_cols = preprocessor.preprocess()

#### Titanic dataset

Load the dataset

In [ ]:
data = sns.load_dataset("titanic")
data

Set the target variable

In [ ]:
target = 'survived'

For using the sequential agent - select the feature by which to sequence the data. This can be skipped if not using the sequential agent. <br/>
In this case, the selected feature is age, as it makes sense for the survival rate of passengers of different ages to be different.

In [ ]:
sequence_by = 'age'

Next, preprocess the data using the preprocessing pipeline.

In [ ]:
preprocessor = PreprocessingPipeline(data, [
    'pclass', 'sex', 'sibsp', 'parch', 'embarked', 
    'class', 'who', 'adult_male', 'deck', 'embark_town', 'alive', 'alone'
], 0.9, target_name=target)
data, continuous_cols, cat_cols = preprocessor.preprocess()

### Model and metric selection
For the classification datasets, we use a logistic regression model, log loss as the loss function, as well as accuracy, precision, recall and F1 score as the evaluation metrics.

In [ ]:
model = LogisticRegression()
loss_func = log_loss
metrics = [accuracy_score, precision_score, recall_score, f1_score]

### Normalization - optional
It is highly recommended to use this normalization step for the classification datasets, as sklearn's LogisticRegression model is sensitive to the scale of the features.<br/>
On the other hand, for the linear regression datasets, this step is entirely optional, as the model used (LinearRegression) is not sensitive to the scale of the features.

In [ ]:
scaler = StandardScaler()
data[continuous_cols] = scaler.fit_transform(data[continuous_cols])

### Split the data

Split the data into training, validation and test sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, data[target], test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=42)

# Feature Selection

## Filter methods

Filter methods are methods that select features based on some statistical measure calculated from the data. They do not involve training a model, and are therefore computationally less expensive than wrapper methods. <br/>
The filter methods implemented in this project are:
1. A weighted sum of correlation and mutual information
2. Variance Inflation Factor (VIF)

Both methods have tunable hyperparameters, which may be adjusted or optimized automatically using the provided methods.<br/>
<br/>
The end of this section contains an evaluation, allowing you to compare the model's performance with and without feature selection using the selected method, as well as compare to sklearn's SelectKBest method.

### Weighted Sum Method

#### Method description

This method computes a weighted sum between the mutual information and the correlation of each feature with the target variable. <br/>
It has a weight parameter $\alpha$, such that the score of each feature is computed as 
$$\alpha \cdot \text{correlation} + (1-\alpha) \cdot \text{mutual information}$$
The value of $\alpha$ is set to 0.5 by default, but can be adjusted by the user.<br/>
<br/>
The algorithm has 2 more hyperparameters - the maximum number of features to select, and a score threshold (features with a score below this threshold are removed).<br/>
Both are set to 0 by default, but at least one of them must be set by the user.

#### Method usage

##### Basic usage

If the target was removed from the data, add it back to the data, as this method requires the target to be present in the data.

In [ ]:
if target not in X_train.columns:
    X_train[target] = y_train
    X_val[target] = y_val
    X_test[target] = y_test

Instantiate the WeightedCombination class, and fit it to the data.

In [ ]:
feature_selector_weighted = WeightedCombination(correlation_weight=0.5)

In [ ]:
feature_selector_weighted.fit(X_train, target_column=target, continuous_cols=continuous_cols,categorical_cols= cat_cols)

Next, you can either run the following cell select features using the current hyperparameters:

In [ ]:
X_train_after_selection, selected_features_weighted, feature_scores = feature_selector_weighted.transform(X_train, num_features=5, min_threshold=0.1)

Or you can use the following cell to run the hyperparameter optimization process, which will run a grid search over the hyperparameters and select the best ones.<br/>
By default, this process will not optimize the score threshold, as it is computationally expensive. If you wish to optimize it, set the optimize_threshold parameter to True.

##### Hyperparameter optimization

In [ ]:
X_transformed, selected_features_weighted, feature_scores, best_weight, best_loss, best_num_features, best_threshold, history = feature_selector_weighted.auto_optimize(X_train, y_train, X_val, y_val, model=model, loss_function=loss_func)

If you ran the optimization process, you also get a history of the optimization process, which you can plot using the following cell.<br/>
This gives a nice visualization of the loss as a function of the hyperparameters, showing if and how the model performs with less features, or with a different weight between correlation and mutual information.

In [ ]:
weights, num_features = np.meshgrid(history.columns, history.index)
plt.scatter(weights, num_features, c=history.values, cmap='viridis')
plt.xlabel('Weight')
plt.ylabel('Number of features')
plt.title('Loss vs. Weight and Number of Features')
plt.colorbar()
plt.show()

##### Features and training a model

Examine the selected features

In [ ]:
print(f'Number of features selected: {len(selected_features_weighted)}')
print(f'Selected features: {selected_features_weighted}')

Train a model using the selected features.

In [ ]:
# Reset the model, to make sure it is not influenced by the previous training
model = model.__class__()
model.fit(X_train[selected_features_weighted], y_train)
y_pred = model.predict(X_test[selected_features_weighted])
loss = loss_func(y_test, y_pred)
with_feature_selection_weighted = {metric.__name__: metric(y_test, y_pred) for metric in metrics}
with_feature_selection_weighted[loss_func.__name__] = loss
with_feature_selection_weighted['Number of features'] = len(selected_features_weighted)
print(f"Metrics with feature selection: {with_feature_selection_weighted}")

### Variance Inflation Factor

#### Method description

Variance Inflation Factor (VIF) is a measure of multicollinearity among the features. It measures how much the variance of the estimated coefficients of a feature is increased due to multicollinearity. <br/>
Generally, a VIF of 1 indicates no multicollinearity, while a VIF of above 5-10 indicates high multicollinearity, and the feature should be removed. <br/>
This method computes the VIF of each feature, and removes all features with a VIF above or below a certain threshold (up to the user's choice). The threshold is set to 5 by default, but can be adjusted by the user or optimized automatically.<br/>
The method also has a hyperparameter to select the maximum number of features to keep, which is set to 0 by default, meaning all features with a VIF below the threshold will be kept.<br/>
<br/>
The user can choose whether the comparison direction is 'above' or 'below' the threshold, meaning whether to remove features with a VIF above or below the threshold. The default is 'above'.<br/>
This is done because depending on the dataset, a high VIF may indicate that the feature contains important information, despite the multicollinearity, and from empirical observations, it seems that some datasets perform much better when removing features with a VIF below the threshold instead of above it.

#### Method usage

##### Basic usage

First, remove the target feature from the data, if it was not already removed.

In [ ]:
if target in X_train.columns:
    X_train = X_train.drop(columns=[target])
    X_val = X_val.drop(columns=[target])
    X_test = X_test.drop(columns=[target])

Instantiate the VarianceInflationFactor class, and fit it to the data.

In [ ]:
feature_selector_vif = VarianceInflationFactor()

In [ ]:
feature_selector_vif.fit(X_train)

Next, you can either run the following cell select features using the current hyperparameters:

In [ ]:
selected_features_vif = feature_selector_vif.predict(X_train, threshold=5, num_features=0, less_than_threshold_comparison=True)

##### Hyperparameter optimization

Or you can use the following cell to run the hyperparameter optimization process, which will run a grid search over the hyperparameters and select the best ones.<br/>

In [ ]:
selected_features_vif, best_loss, best_threshold, best_num_features, best_comparison_direction,history_less_than_comparisons, history_greater_than_comparisons = feature_selector_vif.auto_optimize(X_train, y_train, X_val, y_val, model=model, loss_function=loss_func)

If you ran the optimization process, you also get a history of the optimization process, which you can plot using the following cell.<br/>
This gives a nice visualization of the loss as a function of the hyperparameters, showing if and how the model performs with less features, with a different threshold, or with a different comparison direction.

In [ ]:
thresholds, num_features = np.meshgrid(history_less_than_comparisons.columns, history_less_than_comparisons.index)
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
ax[0].set_xlabel('Threshold')
ax[0].set_ylabel('Number of features')
ax[0].set_title('Loss vs. Threshold and Number of Features (VIF < threshold)')
fig.colorbar(ax[0].scatter(thresholds, num_features, c=history_less_than_comparisons.values, cmap='viridis'))

thresholds, num_features = np.meshgrid(history_greater_than_comparisons.columns, history_greater_than_comparisons.index)
ax[1].set_xlabel('Threshold')
ax[1].set_ylabel('Number of features')
ax[1].set_title('Loss vs. Threshold and Number of Features (VIF > threshold)')
fig.colorbar(ax[1].scatter(thresholds, num_features, c=history_greater_than_comparisons.values, cmap='viridis'))
plt.show()

##### Features and training a model

Examine the selected features

In [ ]:
print(f'Number of features selected: {len(selected_features_vif)}')
print(f'Selected features: {selected_features_vif}')

Train a model using the selected features.

In [ ]:
# Reset the model, to make sure it is not influenced by the previous training
model = model.__class__()
model.fit(X_train[selected_features_vif], y_train)
y_pred = model.predict(X_test[selected_features_vif])
loss = loss_func(y_test, y_pred)
with_feature_selection_vif = {metric.__name__: metric(y_test, y_pred) for metric in metrics}
with_feature_selection_vif[loss_func.__name__] = loss
with_feature_selection_vif['Number of features'] = len(selected_features_vif)
print(f"Metrics with feature selection: {with_feature_selection_vif}")

### Method evaluation

#### Other methods used for comparison

Next, to evaluate the method, we first need to drop the target feature from the data.

In [ ]:
if target in X_train.columns:
    X_train = X_train.drop(columns=[target])
    X_val = X_val.drop(columns=[target])
    X_test = X_test.drop(columns=[target])

Compare to a baseline model, which uses all of the features.

In [ ]:
model = model.__class__()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
loss = loss_func(y_test, y_pred)
without_feature_selection = {metric.__name__: metric(y_test, y_pred) for metric in metrics}
without_feature_selection[loss_func.__name__] = loss
without_feature_selection['Number of features'] = len(X_train.columns)
print(f"Metrics without feature selection: {without_feature_selection}")

Compare to sklearn's VarianceThreshold method (sklearn's model agnostic filter method), with the threshold set to 0.8, same as in the example on the documentation.

In [ ]:
selector = VarianceThreshold(threshold=0.8)
selector.fit(X_train, y_train)
selected_features_sklearn = X_train.columns[selector.get_support()]
print(f'Selected features using VarianceThreshold: {selected_features_sklearn}')

In [ ]:
model = model.__class__()
model.fit(X_train[selected_features_sklearn], y_train)
y_pred = model.predict(X_test[selected_features_sklearn])
loss = loss_func(y_test, y_pred)
with_feature_selection_sklearn_var_thresh = {metric.__name__: metric(y_test, y_pred) for metric in metrics}
with_feature_selection_sklearn_var_thresh[loss_func.__name__] = loss
with_feature_selection_sklearn_var_thresh['Number of features'] = len(selected_features_sklearn)
print(f"Metrics with feature selection using VarianceThreshold: {with_feature_selection_sklearn_var_thresh}")

Finally, compare to sk_learn's SelectKBest as well, using mutual_info_regression as the score function. This method is not model agnostic, but it is still a filter method and can be used for comparison.

In [ ]:
selector = SelectKBest(score_func=mutual_info_regression)
selector.fit(X_train, y_train)
selected_features_sklearn = X_train.columns[selector.get_support()]
print(f'Selected features using SelectKBest: {selected_features_sklearn}')

In [ ]:
model = model.__class__()
model.fit(X_train[selected_features_sklearn], y_train)
y_pred = model.predict(X_test[selected_features_sklearn])
loss = loss_func(y_test, y_pred)
with_feature_selection_sklearn_k_best = {metric.__name__: metric(y_test, y_pred) for metric in metrics}
with_feature_selection_sklearn_k_best[loss_func.__name__] = loss
with_feature_selection_sklearn_k_best['Number of features'] = len(selected_features_sklearn)
print(f"Metrics with feature selection using SelectKBest: {with_feature_selection_sklearn_k_best}")

#### Results

If you only ran the cells for the weighted sum method, run this cell to format the results into a table

In [ ]:
results = pd.DataFrame([with_feature_selection_weighted, without_feature_selection, with_feature_selection_sklearn_var_thresh, with_feature_selection_sklearn_k_best], index=[f'Feature selection: {feature_selector_weighted.__name__}', 'Without feature selection', 'sklearn Variance Threshold', 'sklearn SelectKBest'])
results

If you only ran the cells for the VIF method, run this cell to format the results into a table

In [ ]:
results = pd.DataFrame([with_feature_selection_vif, without_feature_selection, with_feature_selection_sklearn_var_thresh, with_feature_selection_sklearn_k_best], index=[f'Feature selection: {feature_selector_vif.__class__.__name__}', 'Without feature selection', 'sklearn Variance Threshold', 'sklearn SelectKBest'])
results

If you ran both methods, run this cell to format the results into a table

In [ ]:
results = pd.DataFrame([with_feature_selection_weighted, with_feature_selection_vif, without_feature_selection, with_feature_selection_sklearn_var_thresh, with_feature_selection_sklearn_k_best], index=[f'Feature selection: {feature_selector_weighted.__class__.__name__}', f'Feature selection: {feature_selector_vif.__class__.__name__}', 'Without feature selection', 'sklearn Variance Threshold', 'sklearn SelectKBest'])
results

## Wrapper methods using deep reinforcement learning
Wrapper methods are methods that select features by training a model, and using the model's performance as a measure of the feature's importance, treating the model as a black box. <br/>
<br/>
Feature selection can be described as a prediction problem, where a model needs to predict which subset of features will lead to the best performance. <br/>
To do this in an interpretable way using neural networks, there needs to be a "disconnect" between the feature selection model and the model that uses the selected features, thus making it impossible to directly compute gradients for the feature selection model. <br/>
Thus, reinforcement learning was used.<br/>
<br/>
There are 2 methods implemented in this project:
1. LinearAgent
2. SequentialAgent

In the case of these algorithms, the prediction model is used to compute the reward for the feature selection model, and the feature selection model is trained using reinforcement learning. <br/>
<br/>
All methods were implemented using the StableBaselines3 library, which is a PyTorch-based library for reinforcement learning.

### LinearAgent

#### Method description

LinearAgent uses a linear model as a feature extractor for the feature selection task. <br/>
The agent is trained via either the A2C or PPO policies.<br/>
The environment's action space is a binary vector, where each element corresponds to a feature, and the agent can choose to select or not select each feature.<br/>
The environment's observation space is a batch of data.<br/>
Each step the agent predicts features by a batch of data, and the environment returns the reward, which is the performance of the downstream model on the selected features. An episode is a full pass over the data (one epoch).<br/>
<br/>
It uses a default network architecture, where the input layer is the number of features, and each subsequent layer has half the number of neurons as the previous layer, until reaching the size of the output layer (the number of features). Additionally, the network uses ReLU activations and has a layer normalization layer after every 2 layers.<br/>
<br/>
the user can customize Both the network and the environment, by passing the relevant classes to the agent's constructor.<br/>

#### Method usage

##### Training the agent

First, remove the target feature from the data, if it was not already removed.

In [ ]:
if target in X_train.columns:
    X_train = X_train.drop(columns=[target])
    X_val = X_val.drop(columns=[target])
    X_test = X_test.drop(columns=[target])

Set the batch size for the agent

In [ ]:
batch_size = 256

Instantiate the LinearAgent class

In [ ]:
lin_agent = LinearAgent(X_train, y_train, LinearRegression(), mean_squared_error, batch_size=batch_size, agent_type='A2C',save_path="models/linear_agent", eval_freq=500)
print(lin_agent.agent.policy)

Train the agent over a certain number of steps.
Note: this may take a long time.

In [ ]:
lin_agent.learn(num_steps=4000)

Optionally, save the model. This is useful if you want to continue training later, or if you want to load the model for evaluation.<br/>
There is a load method, which can be used to load the model.

In [ ]:
lin_agent.save(model_name="end_of_training")

##### Predicting features and training a model

Predict the features on a batch of data, sampled from the validation data

In [ ]:
X_val_sample = X_val.sample(batch_size, replace=True)

Setting deterministic to True will make the agent always select the same features for the same data, while setting it to False will allow the agent to select a different (potentially similar) set of features for the same data, potentially improving the model's performance.


In [ ]:
action = lin_agent.predict(X_val_sample, deterministic=True)
selected_features = X_train.columns[action == 1]
print(f'Number of selected features: {len(selected_features)}, Selected features: {selected_features}')

Run the cell below to train a model using the selected features.<br/>
The evaluation when compared to other methods can be done further below, beneath the sequential agent section.

In [ ]:
# Reset the model, to make sure it is not influenced by the previous training
model = model.__class__()
model.fit(X_train[selected_features], y_train)
y_pred = model.predict(X_test[selected_features])
loss = loss_func(y_test, y_pred)
with_feature_selection = {metric.__name__: metric(y_test, y_pred) for metric in metrics}
with_feature_selection[loss_func.__name__] = loss
with_feature_selection['Number of features'] = len(selected_features)
print(f"Metrics with feature selection: {with_feature_selection}")

### SequentialAgent

#### Method description

SequentialAgent treats the data as a collection of sequences, where each sequence is a subset of the data, sorted and separated by a certain feature. <br/>
It can use either a linear neural network or a bi-directional self-attentive LSTM network as the feature extractor for the feature selection task with either the A2C or PPO policies, or use the RecurrentPPO policy with a linear feature extractor and a LSTM as the policy network.<br/>
The default uses the A2C policy with a LSTM network as the feature extractor.<br/>
This method can select different features for different sequences (although it may also select the same features for all sequences), which can then be used for training several models, each for a different sequence.<br/>
For example, in the housing dataset, a sequence could be all houses built between 1850 and 1900, and another sequence could be all houses built between 1900 and 1950.<br/>

#### Method usage

##### Training the agent

First, remove the target feature from the data, if it was not already removed.

In [ ]:
if target in X_train.columns:
    X_train = X_train.drop(columns=[target])
    X_val = X_val.drop(columns=[target])
    X_test = X_test.drop(columns=[target])

Instantiate the SequentialAgent class

In [ ]:
sequential_agent = SequentialAgent(
    X_train=X_train,
    y_train=y_train,
    col=sequence_by,
    downstream_model=model,
    loss_function=loss_func,
    agent_type='A2C',
    network_type='recurrent',
    save_path='models/sequential_agent',
    eval_freq=500,
    lstm_hidden_layer_size=64,
    lstm_num_layers=2,
    clustering_method='MeanShift'
)
print(sequential_agent.agent.policy)

It is possible to examine the sequences the model will use for training by using the sequencer object, which can be obtained by calling the get_sequencer method on the agent.

In [ ]:
sequencer = sequential_agent.get_sequencer()
train_sequences, train_targets = sequencer.sequence_by_ranges(X_train, y_train)
print(f'Number of sequences: {len(train_sequences)}')
print(f'Sequence lengths: {[len(sequence) for sequence in train_sequences]}')

Train the agent for a specified number of steps.
Note: this may take a long time.

In [ ]:
sequential_agent.learn(num_steps=2000)

Optionally - save the model

In [ ]:
sequential_agent.save(model_name="end_of_training")

##### Predicting features and training models

Get the agent's predictions on the validation set, then train models on the predicted feature set.

In [ ]:
predictions = sequential_agent.predict(X_train, deterministic=True)
for prediction in predictions:
    print(X_train.columns[prediction == 1])

Initialize a list of models, then sequence the training and test data by the ranges of sequences found on the training set.

In [ ]:
models_with_sequencing_and_feature_selection = []
metrics_with_sequencing_and_feature_selection = []
sequencer = sequential_agent.get_sequencer()
train_sequences, train_targets = sequencer.sequence_by_ranges(X_train, y_train)
test_sequences, test_targets = sequencer.sequence_by_ranges(X_test, y_test)

In [ ]:
for i in range(len(train_sequences)):
    X_train_sequence = train_sequences[i]
    # Choose the features selected by the agent
    X_train_sequence = X_train_sequence[X_train_sequence.columns[predictions[i] == 1]]
    y_train_sequence = train_targets[i]
    X_test_sequence = test_sequences[i]
    X_test_sequence = X_test_sequence[X_test_sequence.columns[predictions[i] == 1]]
    y_test_sequence = test_targets[i]
    
    # If any sequence is empty, we skip training a model on it. An empty sequence indicates that any samples in the sequence range are outliers.
    # This is an edge case, which should normally not happen, but can happen if the dataset has either a very small number of samples or a small number of outliers that could not be grouped together.
    if X_train_sequence.shape[0] == 0 or X_test_sequence.shape[0] == 0:
        continue
    
    model = model.__class__()
    model.fit(X_train_sequence, y_train_sequence)
    y_pred = model.predict(X_test_sequence)
    
    loss = loss_func(y_test_sequence, y_pred)
    sequence_metrics = {metric.__name__: metric(y_test_sequence, y_pred) for metric in metrics}
    sequence_metrics[loss_func.__name__] = loss
    sequence_metrics['Number of features'] = len(X_train_sequence.columns)
    
    metrics_with_sequencing_and_feature_selection.append(sequence_metrics)
    models_with_sequencing_and_feature_selection.append(model)

In [ ]:
for metric_collection in metrics_with_sequencing_and_feature_selection:
    print(metric_collection)

Compute and print average, max and min metrics along all of the models

In [ ]:
avg_metrics_with_sequencing_with_feature_selection = {metric: np.mean([model_metrics[metric] for model_metrics in metrics_with_sequencing_and_feature_selection]) for metric in
                   metrics_with_sequencing_and_feature_selection[0].keys()}
max_metrics_with_sequencing_with_feature_selection = {metric: np.max([model_metrics[metric] for model_metrics in metrics_with_sequencing_and_feature_selection]) for metric in
                   metrics_with_sequencing_and_feature_selection[0].keys()}
min_metrics_with_sequencing_with_feature_selection = {metric: np.min([model_metrics[metric] for model_metrics in metrics_with_sequencing_and_feature_selection]) for metric in
                   metrics_with_sequencing_and_feature_selection[0].keys()}
print(f"Average metrics: {avg_metrics_with_sequencing_with_feature_selection}")
print(f"Max metrics: {max_metrics_with_sequencing_with_feature_selection}")
print(f"Min metrics: {min_metrics_with_sequencing_with_feature_selection}")

For comparison, train a model with the same sequences, but without feature selection.

In [ ]:
models_with_sequencing_and_without_feature_selection = []
metrics_with_sequencing_and_without_feature_selection = []

In [ ]:
for i in range(len(train_sequences)):
    X_train_sequence = train_sequences[i]
    y_train_sequence = train_targets[i]
    X_test_sequence = test_sequences[i]
    y_test_sequence = test_targets[i]
    
    # If any sequence is empty, we skip training a model on it. An empty sequence indicates that any samples in the sequence range are outliers.
    # This is an edge case, which should normally not happen, but can happen if the dataset has either a very small number of samples or a small number of outliers that could not be grouped together.
    if X_train_sequence.shape[0] == 0 or X_test_sequence.shape[0] == 0:
        continue
    
    model = model.__class__()
    model.fit(X_train_sequence, y_train_sequence)
    y_pred = model.predict(X_test_sequence)
    
    loss = loss_func(y_test_sequence, y_pred)
    sequence_metrics = {metric.__name__: metric(y_test_sequence, y_pred) for metric in metrics}
    sequence_metrics[loss_func.__name__] = loss
    sequence_metrics['Number of features'] = len(X_train_sequence.columns)
    
    metrics_with_sequencing_and_without_feature_selection.append(sequence_metrics)
    models_with_sequencing_and_without_feature_selection.append(model)

In [ ]:
for metric_collection in metrics_with_sequencing_and_without_feature_selection:
    print(metric_collection)

In [ ]:
avg_metrics_with_sequencing_without_feature_selection = {metric: np.mean([model_metrics[metric] for model_metrics in metrics_with_sequencing_and_without_feature_selection]) for metric in
                   metrics_with_sequencing_and_without_feature_selection[0].keys()}
max_metrics_with_sequencing_without_feature_selection = {metric: np.max([model_metrics[metric] for model_metrics in metrics_with_sequencing_and_without_feature_selection]) for metric in
                     metrics_with_sequencing_and_without_feature_selection[0].keys()}
min_metrics_with_sequencing_without_feature_selection = {metric: np.min([model_metrics[metric] for model_metrics in metrics_with_sequencing_and_without_feature_selection]) for metric in
                        metrics_with_sequencing_and_without_feature_selection[0].keys()}
print(f"Average metrics: {avg_metrics_with_sequencing_without_feature_selection}")
print(f"Max metrics: {max_metrics_with_sequencing_without_feature_selection}")
print(f"Min metrics: {min_metrics_with_sequencing_without_feature_selection}")

#### Comparative method evaluation

#### Other methods used for comparison

Compare to a baseline model, which uses all of the features

In [ ]:
model = model.__class__()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
loss = loss_func(y_test, y_pred)
without_feature_selection = {metric.__name__: metric(y_test, y_pred) for metric in metrics}
without_feature_selection[loss_func.__name__] = loss
without_feature_selection['Number of features'] = len(X_train.columns)
print(f"Metrics without feature selection: {without_feature_selection}")

Compare to sklearn's Sequential Feature Selector, a wrapper method that chooses features in an iterative process, with the default settings.

In [ ]:
selector = SequentialFeatureSelector(model.__class__())
selector.fit(X_train, y_train)
selected_features_sklearn = X_train.columns[selector.support_]
print(f'Selected features using Sequential Feature Selector: {selected_features_sklearn}')

In [ ]:
model = model.__class__()
model.fit(X_train[selected_features_sklearn], y_train)
y_pred = model.predict(X_test[selected_features_sklearn])
loss = loss_func(y_test, y_pred)
with_feature_selection_sklearn = {metric.__name__: metric(y_test, y_pred) for metric in metrics}
with_feature_selection_sklearn[loss_func.__name__] = loss
with_feature_selection_sklearn['Number of features'] = len(selected_features_sklearn)
print(f"Metrics with feature selection using Sequential Feature Selector: {with_feature_selection_sklearn}")

#### Results

If you used the linear agent, run this cell to format the results into a table

In [ ]:
results = pd.DataFrame([with_feature_selection, without_feature_selection, with_feature_selection_sklearn], index=[f'Feature selection: {lin_agent.__class__.__name__}', 'Without feature selection', 'Sequential Feature Selector'])
results

If you used the sequential agent, run this cell to format the results into a table

In [ ]:
results = pd.DataFrame([avg_metrics_with_sequencing_with_feature_selection, max_metrics_with_sequencing_with_feature_selection, min_metrics_with_sequencing_with_feature_selection, avg_metrics_with_sequencing_without_feature_selection, max_metrics_with_sequencing_without_feature_selection, min_metrics_with_sequencing_without_feature_selection, without_feature_selection, with_feature_selection_sklearn], index=['Average metrics with sequencing and feature selection', 'Max metrics with sequencing and feature selection', 'Min metrics with sequencing and feature selection', 'Average metrics with sequencing and without feature selection', 'Max metrics with sequencing and without feature selection', 'Min metrics with sequencing and without feature selection', 'Without feature selection', 'Sequential Feature Selector'])
results

## Wrapper methods using SHAP values

This section introduces two variants of the algorithm presented in the paper "A feature selection method based on Shapley values robust to
    concept shift in regression" by Carlos Sebastián and Carlos E. González-Guillén, arXiv preprint arXiv:2304.14774 (2023) (https://arxiv.org/abs/2304.14774).<br/>
<br/>
The algorithm presented in the paper is composed of 2 phases:
1. The optional preliminary phase
2. The main phase

In the optional preliminary phase, the algorithm introduces a random variable to the data, then removes all features that have less influence on the model than the random variable.<br/>
This can help to greatly reduce the number of features while losing minimal information. However, if too many features are removed, the model may lose important information, and the model's performance may decrease.<br/>
<br/>
In the main phase of the algorithm, the algorithm sequentially removes the feature that has the highest negative influence on the model among the currently selected features.<br/>
This iterates until either a desired number of features are removed or until there are no more features to remove.<br/>
At the end of the main phase, the algorithm returns the best subset of features found during the process.<br/>
<br/>
For the full and in-depth explanation of the algorithm, please refer to the paper.<br/>
<br/>
The algorithm was implemented in this project, with 2 variants that modify the main phase:
1. Branching
2. Backtracking

Before procedding, drop the target feature from the data, if it was not already removed.

In [ ]:
if target in X_train.columns:
    X_train = X_train.drop(columns=[target])
    X_val = X_val.drop(columns=[target])
    X_test = X_test.drop(columns=[target])

Also, run this cell to initialize the metric dictionaries for all of the methods.
This will be used later to create a table, without having to run the evaluation cells for each method.

In [ ]:
with_feature_selection_base = {metric.__name__: np.nan for metric in metrics}
with_feature_selection_branching = {metric.__name__: np.nan for metric in metrics}
with_feature_selection_backtracking = {metric.__name__: np.nan for metric in metrics}

### Base method

The base method is an implementation of the algorithm described in the paper.<br/>
It can be used as follows:

Instantiate the BaseMethod class and predict the features.<br/>
The hyperparamters $q_{low}$ and $q_{high}$ are used for quantile selection. For the exact explanation of these hyperparameters, please refer to the paper.

In [ ]:
base_method = BaseMethod(q_low=0.15, q_high=0.85)

In [ ]:
base_method_features, best_score = base_method.predict(X_train, y_train, X_val, y_val, model, loss_func, num_iter_prev=1)

Observe the selected features

In [ ]:
print(f"Number of features: {len(base_method_features)}")
print(f"Features: {base_method_features}")

Train a model using the selected features.

In [ ]:
model = model.__class__()
model.fit(X_train[base_method_features], y_train)
y_pred = model.predict(X_test[base_method_features])
loss = loss_func(y_test, y_pred)
with_feature_selection_base = {metric.__name__: metric(y_test, y_pred) for metric in metrics}
with_feature_selection_base[loss_func.__name__] = loss
with_feature_selection_base['Number of features'] = len(base_method_features)
print(f"Metrics with feature selection: {with_feature_selection_base}")

### Branching variant

#### Method description

In the branching variant, the algorithm creates 2 branches at each iteration of the main phase, one where the feature with the highest negative influence is removed, and one where the feature with the second highest negative influence is removed, thus allowing the algorithm to explore much more of the search space.<br/>
This can lead to better results, but may also be more computationally expensive.<br/> 

#### Method usage

Instantiate the BranchingVariant class and predict the best feature subset.<br/>
Note: this may take some time to run, particularly so on datasets with a large number of features.

In [ ]:
branching_method = BranchingVariant(q_low=0.15, q_high=0.85)

In [ ]:
branching_method_features, best_score = branching_method.predict(X_train, y_train, X_val, y_val, model, loss_func, num_iter_prev=1)

Observe the selected features

In [ ]:
print(f"Number of features: {len(branching_method_features)}")
print(f"Features: {branching_method_features}")

Train a model using the selected features.

In [ ]:
model = model.__class__()
model.fit(X_train[branching_method_features], y_train)
y_pred = model.predict(X_test[branching_method_features])
loss = loss_func(y_test, y_pred)
with_feature_selection_branching = {metric.__name__: metric(y_test, y_pred) for metric in metrics}
with_feature_selection_branching[loss_func.__name__] = loss
with_feature_selection_branching['Number of features'] = len(branching_method_features)
print(f"Metrics with feature selection: {with_feature_selection_branching}")


### Backtracking variant

#### Method description

The algorithm described in the paper is a Sequential Feature Selection (SFS) algorithm.<br/>
This variant transforms it into a Sequential Floating Forward Selection (SFFS) algorithm, by adding a backtracking mechanism.<br/>
The backtracking mechanism allows the algorithm to add previously removed features back to the selected features, if it improves the model's performance.<br/>

#### Method usage

Instantiate the BacktrackingVariant class and predict the best feature subset.<br/>
Note: this may take some time to run, particularly so on datasets with a large number of features.

In [ ]:
backtracking_method = BacktrackingVariant(q_low=0.15, q_high=0.85)

In [ ]:
backtracking_method_features, best_score = backtracking_method.predict(X_train, y_train, X_val, y_val, model, loss_func, num_iter_prev=1)

Observe the selected features

In [ ]:
print(f"Number of features: {len(backtracking_method_features)}")
print(f"Features: {backtracking_method_features}")

Train a model using the selected features.

In [ ]:
model = model.__class__()
model.fit(X_train[backtracking_method_features], y_train)
y_pred = model.predict(X_test[backtracking_method_features])
loss = loss_func(y_test, y_pred)
with_feature_selection_backtracking = {metric.__name__: metric(y_test, y_pred) for metric in metrics}
with_feature_selection_backtracking[loss_func.__name__] = loss
with_feature_selection_backtracking['Number of features'] = len(backtracking_method_features)
print(f"Metrics with feature selection: {with_feature_selection_backtracking}")

### Method evaluation

#### Other methods used for comparison

Compare to a baseline model, which uses all of the features

In [ ]:
model = model.__class__()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
loss = loss_func(y_test, y_pred)
without_feature_selection = {metric.__name__: metric(y_test, y_pred) for metric in metrics}
without_feature_selection[loss_func.__name__] = loss
without_feature_selection['Number of features'] = len(X_train.columns)
print(f"Metrics without feature selection: {without_feature_selection}")

Compare to sklearn's Sequential Feature Selection method, with the direction set to backward, to make it as close in behavior to the algorithm as possible.

In [ ]:
selector = SequentialFeatureSelector(model.__class__(), direction='backward')
selector.fit(X_train, y_train)
selected_features_sklearn = X_train.columns[selector.support_]
print(f'Selected features using Sequential Feature Selector: {selected_features_sklearn}')

In [ ]:
model = model.__class__()
model.fit(X_train[selected_features_sklearn], y_train)
y_pred = model.predict(X_test[selected_features_sklearn])
loss = loss_func(y_test, y_pred)
with_feature_selection_sklearn = {metric.__name__: metric(y_test, y_pred) for metric in metrics}
with_feature_selection_sklearn[loss_func.__name__] = loss
with_feature_selection_sklearn['Number of features'] = len(selected_features_sklearn)
print(f"Metrics with feature selection using Sequential Feature Selector: {with_feature_selection_sklearn}")

#### Results

Format the results in a table.

In [ ]:
results = pd.DataFrame([with_feature_selection_base, with_feature_selection_branching, with_feature_selection_backtracking, without_feature_selection, with_feature_selection_sklearn], index=['Base method', 'Branching variant', 'Backtracking variant', 'Without feature selection', 'sklearn Sequential Feature Selector (backwards direction)'])
results